In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np

from datetime import datetime

종목코드열람

In [2]:
# SQLite DB 연결
conn = sqlite3.connect("t8430.db")
# Connection 으로부터 Cursor 생성
cur = conn.cursor()
# SQL 쿼리 실행
query = "select distinct 종목명, 종목코드 \
        from 종목코드 ;"
        #where 종목코드 = '%s'; " % '000020'
cur.execute(query)
# 데이타 Fetch
rows = cur.fetchall()
shcode_list = pd.DataFrame(rows, columns = ['종목명', '종목코드'])
# Connection 닫기
conn.close()

시세데이터 뽑기

In [3]:
def query_ohlcv(shcode):
    # SQLite DB 연결
    conn = sqlite3.connect("t1305_1.db")
    # Connection 으로부터 Cursor 생성
    cur = conn.cursor()
    # SQL 쿼리 실행
    query = "select 일자, 시가, 고가, 저가, 종가, 누적거래량 \
            from 종목별체결조회 \
            where 1=1 \
            and 일자 between '20190101' and '20190927'\
            and 종목코드 = '%s' \
            and 일주월구분 = '%s' ;" % (shcode,'1')
    cur.execute(query)
    # 데이타 Fetch
    query_result = cur.fetchall()
    fin_result = pd.DataFrame(query_result, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    # Connection 닫기
    daily = fin_result.sort_values(by = ['date']).reset_index(drop = True)
    conn.close()
    return daily

In [16]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [17]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [18]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 5000000000

for i, v in enumerate(shcode_list['종목코드']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['종목명'].iloc[i]))
    
    daily_data = query_ohlcv(v)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = np.where(result_test==1)
    final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[i,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['종목명'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '.csv'
result.to_csv(filename, sep=',')

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 10000000000

for i, v in enumerate(shcode_list['종목코드']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['종목명'].iloc[i]))
    
    daily_data = query_ohlcv(v)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))       
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = np.where(result_test==1)
    final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[i,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['종목명'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '.csv'
result.to_csv(filename, sep=',')

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['종목코드']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['종목명'].iloc[i]))
    
    daily_data = query_ohlcv(v)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = np.where(result_test==1)
    final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[i,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['종목명'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '.csv'
result.to_csv(filename, sep=',')